<center><font size=5>单因子示例</font></center>

目前只有IC相关的计算。余下的有待大家探索。

加载各种包

In [1]:
import matplotlib.pyplot as plt
import sys
sys.path.append('..')  # 文件位于

import factors  # 因子计算
import data_utils  # 数据读取
import performance as perf  # IC计算
import plotting  #绘图

# 1. 初始化DataLoader 

DataLoader实现了对csv文件的批量读取，会按照\[年\]\[月\]保存在dl中。
一些重要函数：
- init_month_df: 加载目标文件夹下的所有zip文件。这些文件读取后会被保存在dataloader中。注意，

In [2]:
dl = data_utils.DataLoader()
dl.init_month_df('../../data/UDST_M/15m_data/')
# 计算未来收益率
data_utils.compute_future_rtn_for_all(dl)

In [3]:
import pandas as pd
import numpy as np

In [4]:
def cal_Z_score(factor_df, column_lists, span):
    '''Z-score'''
    for column_name in column_lists:
        mean_20 = factor_df[column_name].ewm(span, adjust=False).mean()
        std_20 = factor_df[column_name].ewm(96, adjust=False).std()  # todo 超参数搜索
        factor_df[f'{column_name}_Z'] = (factor_df[column_name].values - mean_20) / std_20
    return factor_df

@factors.handle_dataloader_data
def add_close_Z(raw_df:pd.DataFrame)-> pd.DataFrame:
    raw_df = cal_Z_score(raw_df, ['close'], 30)
    return raw_df

# pandas_ta 因子

## Statistics (11)

* _Entropy_: **entropy**
* _Kurtosis_: **kurtosis**
* _Mean Absolute Deviation_: **mad**
* _Median_: **median**
* _Quantile_: **quantile**
* _Skew_: **skew**
* _Standard Deviation_: **stdev**
* _Think or Swim Standard Deviation All_: **tos_stdevall**
* _Variance_: **variance**
* _Z Score_: **zscore**

In [5]:
from pandas_ta import statistics

In [28]:
@factors.handle_dataloader_data
def add_new_factor(raw_df:pd.DataFrame)-> pd.DataFrame:
    '''
    加入新因子的模板。
    :param raw_df: 原始数据
    :param factor_name: 因子名称
    :return: 原始数据+因子
    '''
    l = 100
    try:
        data_df = raw_df[['open', 'close', 'high', 'low', 'volume','taker_buy_volume','fwd_rtn_1_real']]
        data_df.index = pd.to_datetime(raw_df.open_time, unit='ms')
        temp_factor_df = pd.DataFrame()  # 暂存特征，避免index不一致导致的错误
        data_df['zscore']   = statistics.zscore(data_df['close'], length=20)
        temp_factor_df['zscore']   = data_df['zscore']
        temp_factor_df['kurtosis'] = statistics.kurtosis(data_df['fwd_rtn_1_real'], length=l)
        temp_factor_df['mad']      = statistics.mad(data_df['fwd_rtn_1_real'], length=l)
        temp_factor_df['median']   = statistics.median(data_df['fwd_rtn_1_real'], length=l)
        temp_factor_df['quantile'] = statistics.quantile(data_df['fwd_rtn_1_real'], length=l)
        temp_factor_df['skew']     = statistics.skew(data_df['fwd_rtn_1_real'], length=l)
        temp_factor_df['stdev']    = statistics.stdev(data_df['fwd_rtn_1_real'], length=l)
        temp_factor_df['variance'] = statistics.variance(data_df['fwd_rtn_1_real'], length=l)
        temp_factor_df.reset_index(drop=True, inplace=True)
        raw_df = pd.concat([raw_df, temp_factor_df], axis=1)
        return raw_df
    except:
        return pd.DataFrame()

In [29]:
# 计算因子
add_new_factor(dl)

F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\pandas\core\window\rolling.py:525: RuntimeWarning:

All-NaN slice encountered



In [ ]:
add_close_Z(dl)

In [ ]:
from ml_performance import MLFactorEvaluator

In [ ]:
record_names = ['baseline', 'setA']

In [ ]:
factor_names = [
    ['zscore'],
    ['kurtosis', 'mad', 'median', 'quantile', 'skew', 'stdev', 'variance', 'zscore']
]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from xgboost import XGBClassifier

In [ ]:
params = {
    'n_jobs' : -1,
    'max_depth' : 5,
    'n_estimators' : 30,
#     'reg_alpha' : 0.4,
}

In [ ]:
ml_evaluator = MLFactorEvaluator(
    method = RandomForestClassifier, 
    record_names = record_names, 
    factor_names = factor_names,
    params_dict = params
)

In [ ]:
ml_evaluator.evaluate(dl, 'fwd_rtn_20_bool')

In [ ]:
ml_record = ml_evaluator.get_assess_df('f1')

In [ ]:
plotting.plot_ML_table(ml_record)

In [ ]:
plotting.plot_ML_records(ml_record)

In [ ]:
test_df = dl.get_month_df('2021','10')[0]

In [ ]:
test_df = test_df.dropna()

In [ ]:
import cufflinks

In [ ]:
test_df[['kurtosis', 'mad', 'median', 'quantile', 'skew', 'stdev', 'variance', 'zscore']].iplot(kind='box')

In [16]:
def check_validation(df:pd.DataFrame):
    '''检查数据是不是有效'''
    df_length = len(df)
    if len(df.dropna()) == 0:
        return False
    else:
        return True

In [19]:
valid_list = dl.get_valid_year_month()
for year, month in valid_list:
    temp_df_list = dl.get_month_df(year, month)
    for i, df in enumerate(temp_df_list):
        valid = check_validation(df)
        if not valid:
            print(f'year:{year}, month:{month}, i:{i}, is not valid')

year:2020, month:01, i2, is not valid
year:2020, month:06, i5, is not valid
year:2021, month:03, i28, is not valid
year:2022, month:07, i31, is not valid
year:2022, month:08, i31, is not valid
year:2022, month:12, i29, is not valid
year:2023, month:01, i30, is not valid
year:2023, month:02, i31, is not valid
year:2023, month:03, i32, is not valid
year:2023, month:04, i25, is not valid
year:2023, month:04, i33, is not valid
year:2023, month:05, i33, is not valid
year:2023, month:06, i34, is not valid
year:2023, month:07, i36, is not valid
year:2023, month:08, i38, is not valid
year:2023, month:09, i40, is not valid
year:2023, month:10, i41, is not valid


In [20]:
test_df = dl.get_month_df('2020','01')[2]

In [26]:
test_df.shape

(63, 38)

In [23]:
test_df.isna().sum()

open_time                  0
open                       0
high                       0
low                        0
close                      0
volume                     0
close_time                 0
quote_volume               0
count                      0
taker_buy_volume           0
taker_buy_quote_volume     0
ignore                     0
fwd_rtn_5                  5
fwd_rtn_5_real             9
fwd_rtn_5_bool             0
fwd_rtn_10                10
fwd_rtn_10_real           14
fwd_rtn_10_bool            0
fwd_rtn_20                20
fwd_rtn_20_real           24
fwd_rtn_20_bool            0
fwd_rtn_50                50
fwd_rtn_50_real           54
fwd_rtn_50_bool            0
fwd_rtn_100               63
fwd_rtn_100_real          63
fwd_rtn_100_bool           0
fwd_rtn_1                  1
fwd_rtn_1_real             5
fwd_rtn_1_bool             0
zscore                    19
kurtosis                  63
mad                       63
median                    63
quantile      

In [21]:
len(test_df.dropna())

0

In [15]:
len(test_df)

2975

In [27]:
pd.concat([test_df, pd.DataFrame()])

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,...,fwd_rtn_1_real,fwd_rtn_1_bool,zscore,kurtosis,mad,median,quantile,skew,stdev,variance
0,1580458500000,0.05386,0.05404,0.05374,0.05393,2154869,1580459399999,116140.60669,1811,1338371,...,NaN,True,NaN,None,None,None,None,None,None,None
1,1580459400000,0.05393,0.05484,0.05386,0.05448,5057082,1580460299999,274981.55592,942,2921377,...,NaN,True,NaN,None,None,None,None,None,None,None
2,1580460300000,0.05448,0.05494,0.05431,0.05458,3089624,1580461199999,168650.55881,747,1684088,...,NaN,True,NaN,None,None,None,None,None,None,None
3,1580461200000,0.05459,0.05539,0.05457,0.05525,3704440,1580462099999,203682.18638,1030,2402933,...,NaN,False,NaN,None,None,None,None,None,None,None
4,1580462100000,0.05524,0.05559,0.05465,0.05481,6240834,1580462999999,344513.97519,1025,3366330,...,-0.379964,False,NaN,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,1580510700000,0.05431,0.05458,0.05417,0.05423,4908810,1580511599999,266915.21068,499,2031665,...,0.207080,True,1.697403,None,None,None,None,None,None,None
59,1580511600000,0.05421,0.05430,0.05411,0.05429,2804548,1580512499999,152073.47246,402,1439541,...,-0.761607,False,1.619351,None,None,None,None,None,None,None
60,1580512500000,0.05426,0.05430,0.05402,0.05407,3391592,1580513399999,183599.56861,435,1714293,...,0.070304,True,0.994057,None,None,None,None,None,None,None
61,1580513400000,0.05406,0.05429,0.05402,0.05409,2330371,1580514299999,126065.58338,371,1155337,...,-0.776221,False,0.944059,None,None,None,None,None,None,None
